In [6]:
import os
machine_learning_api_script = os.path.join(os.path.pardir, "src" ,"models", 'machine_learning_api.py') 

In [106]:
%%writefile $machine_learning_api_script

from flask import Flask,request
import pandas as pd
import os
import numpy as np
import pickle
import json

app =Flask(__name__)

model_path = os.path.join(os.path.pardir,os.path.pardir,'models')

model_file_path = os.path.join(model_path, 'lr_model.pkl')

saclar_file_path = os.path.join(model_path, 'lr_scaler.pkl')

#Import Model
with open(model_file_path, 'rb') as file:
    model = pickle.load(file)

with open(saclar_file_path, 'rb') as file:
    scalar = pickle.load(file)

# columns
columns = [ u'Age', u'Fare', u'FamilySize', \
       u'IsMother', u'IsMale', u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', \
       u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2', \
       u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss', u'Title_Mr', \
       u'Title_Mrs', u'Title_Officer', u'Title_Sir', u'Fare_Bin_very_low', \
       u'Fare_Bin_low', u'Fare_Bin_high', u'Fare_Bin_very_high', u'Embarked_C', \
       u'Embarked_Q', u'Embarked_S', u'AgeState_Adult', u'AgeState_Child'] 

#prediction api
@app.route('/api', methods=['POST'])
def make_prediction():
    # read json object and conver to json string
    data = request.get_json()
    # create pandas dataframe using json string
    df = pd.read_json(data)
    # extract passengerIds
    #passenger_ids = df['PassengerId'].ravel()
    # actual survived values
    actuals = df['Survived'].ravel()
    # extract required columns based and convert to matrix
    X = df[columns].values.astype('float')
    # transform the input
    X_scaled = scalar.transform(X)
    # make predictions
    predictions = model.predict(X_scaled)
    # create response dataframe
    df_response = pd.DataFrame({'Predicted' : predictions, 'Actual' : actuals})
    # return json 
    return df_response.to_json()

if __name__ == '__main__':
     app.run(host = '0.0.0.0', port = 10001, debug=False)
    


Overwriting ..\src\models\machine_learning_api.py


## Call the API

In [24]:
import os
import pandas as pd

In [25]:
#create data

processed_data_path = os.path.join(os.path.pardir, 'data','processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')

df = pd.read_csv(train_file_path, index_col = 'PassengerId')



In [87]:
survived_passengers = df[df.Survived == 1][:2]

# 5 survived passengers
survived_passengers.to_json(orient='index')

'{"2":{"Survived":1,"Age":38.0,"Fare":71.2833,"FamilySize":2,"IsMother":0,"IsMale":0,"Deck_A":0,"Deck_B":0,"Deck_C":1,"Deck_D":0,"Deck_E":0,"Deck_F":0,"Deck_G":0,"Deck_Z":0,"Pclass_1":1,"Pclass_2":0,"Pclass_3":0,"Title_Lady":0,"Title_Master":0,"Title_Miss":0,"Title_Mr":0,"Title_Mrs":1,"Title_Officer":0,"Title_Sir":0,"Fare_Bin_very_low":0,"Fare_Bin_low":0,"Fare_Bin_high":0,"Fare_Bin_very_high":1,"Embarked_C":1,"Embarked_Q":0,"Embarked_S":0,"AgeState_Adult":1,"AgeState_Child":0},"3":{"Survived":1,"Age":26.0,"Fare":7.925,"FamilySize":1,"IsMother":0,"IsMale":0,"Deck_A":0,"Deck_B":0,"Deck_C":0,"Deck_D":0,"Deck_E":0,"Deck_F":0,"Deck_G":0,"Deck_Z":1,"Pclass_1":0,"Pclass_2":0,"Pclass_3":1,"Title_Lady":0,"Title_Master":0,"Title_Miss":1,"Title_Mr":0,"Title_Mrs":0,"Title_Officer":0,"Title_Sir":0,"Fare_Bin_very_low":0,"Fare_Bin_low":1,"Fare_Bin_high":0,"Fare_Bin_very_high":0,"Embarked_C":0,"Embarked_Q":0,"Embarked_S":1,"AgeState_Adult":1,"AgeState_Child":0}}'

## Make Api call

In [27]:
import requests

In [60]:
def make_api_requets(data):
    post_url = 'http://127.0.0.1:5000/api/v1'
    r = requests.post(url=post_url, json=data.to_json())
    return r.json()  

In [105]:
#print(make_api_requets(survived_passengers.all())
post_url = 'http://127.0.0.1:10001/api'
r = requests.post(url=post_url, json=survived_passengers.to_json())
pdf = pd.read_json(json.dumps(r.json()))

pdf.head(5)


,Predicted,Actual
0,1,1
1,1,1


In [103]:
post_url = 'http://127.0.0.1:10001/api'
r = requests.post(url=post_url, json=df.to_json())


pdf = pd.read_json(json.dumps(r.json()))

pdf.head(5)

,Predicted,Actual
0,0,0
1,1,1
10,0,1
100,1,0
101,0,0


In [104]:
np.mean(pdf.Actual == pdf.Predicted)

0.8383838383838383